# Uber Data → Star Schema (Rewritten Notebook)

This notebook is a cleaned, refactored version of the original, keeping the **same input** (`data/uber_data.csv`)
and producing the **same key outputs** as Pandas DataFrames:

- `df`
- `datetime_dim`
- `passenger_count_dim`
- `trip_distance_dim`
- `rate_code_dim`
- `pickup_location_dim`
- `dropoff_location_dim`
- `payment_type_dim`
- `fact_table`


In [ ]:
import pandas as pd

DATA_PATH = "data/uber_data.csv"


## Load data

In [ ]:
df = pd.read_csv(DATA_PATH)
df.head()


In [ ]:
df.info()


## Basic type fixes (same as original)

In [ ]:
df["tpep_pickup_datetime"] = pd.to_datetime(df["tpep_pickup_datetime"])
df["tpep_dropoff_datetime"] = pd.to_datetime(df["tpep_dropoff_datetime"])

df.info()


## Helper + lookup maps

In [ ]:
def build_dim(df: pd.DataFrame, cols: list[str], id_name: str) -> pd.DataFrame:
    """Create a de-duplicated dimension table with a deterministic integer id."""
    dim = df[cols].drop_duplicates().reset_index(drop=True)
    dim[id_name] = dim.index
    return dim

RATE_CODE_TYPE = {
    1: "Standard rate",
    2: "JFK",
    3: "Newark",
    4: "Nassau or Westchester",
    5: "Negotiated fare",
    6: "Group ride",
}

PAYMENT_TYPE_NAME = {
    1: "Credit card",
    2: "Cash",
    3: "No charge",
    4: "Dispute",
    5: "Unknown",
    6: "Voided trip",
}


## Build `datetime_dim` (same columns & ids)

In [ ]:
datetime_dim = build_dim(df, ["tpep_pickup_datetime", "tpep_dropoff_datetime"], "datetime_id")

# Pickup breakdown
datetime_dim["pick_hour"] = datetime_dim["tpep_pickup_datetime"].dt.hour
datetime_dim["pick_day"] = datetime_dim["tpep_pickup_datetime"].dt.day
datetime_dim["pick_month"] = datetime_dim["tpep_pickup_datetime"].dt.month
datetime_dim["pick_year"] = datetime_dim["tpep_pickup_datetime"].dt.year
datetime_dim["pick_weekday"] = datetime_dim["tpep_pickup_datetime"].dt.weekday

# Dropoff breakdown
datetime_dim["drop_hour"] = datetime_dim["tpep_dropoff_datetime"].dt.hour
datetime_dim["drop_day"] = datetime_dim["tpep_dropoff_datetime"].dt.day
datetime_dim["drop_month"] = datetime_dim["tpep_dropoff_datetime"].dt.month
datetime_dim["drop_year"] = datetime_dim["tpep_dropoff_datetime"].dt.year
datetime_dim["drop_weekday"] = datetime_dim["tpep_dropoff_datetime"].dt.weekday

# Column order matches the original notebook
datetime_dim = datetime_dim[
    [
        "datetime_id",
        "tpep_pickup_datetime",
        "pick_hour",
        "pick_day",
        "pick_month",
        "pick_year",
        "pick_weekday",
        "tpep_dropoff_datetime",
        "drop_hour",
        "drop_day",
        "drop_month",
        "drop_year",
        "drop_weekday",
    ]
]

datetime_dim


## Build `passenger_count_dim` and `trip_distance_dim`

In [ ]:
passenger_count_dim = build_dim(df, ["passenger_count"], "passenger_count_id")[
    ["passenger_count_id", "passenger_count"]
]

trip_distance_dim = build_dim(df, ["trip_distance"], "trip_distance_id")[
    ["trip_distance_id", "trip_distance"]
]

passenger_count_dim.head()


In [ ]:
trip_distance_dim.head()


## Build `rate_code_dim`

In [ ]:
rate_code_dim = build_dim(df, ["RatecodeID"], "rate_code_id")
rate_code_dim["rate_code_name"] = rate_code_dim["RatecodeID"].map(RATE_CODE_TYPE)
rate_code_dim = rate_code_dim[["rate_code_id", "RatecodeID", "rate_code_name"]]

rate_code_dim.head()


## Build pickup/dropoff location dimensions

In [ ]:
pickup_location_dim = build_dim(df, ["pickup_longitude", "pickup_latitude"], "pickup_location_id")
pickup_location_dim = pickup_location_dim[["pickup_location_id", "pickup_latitude", "pickup_longitude"]]

dropoff_location_dim = build_dim(df, ["dropoff_longitude", "dropoff_latitude"], "dropoff_location_id")
dropoff_location_dim = dropoff_location_dim[["dropoff_location_id", "dropoff_latitude", "dropoff_longitude"]]


## Build `payment_type_dim`

In [ ]:
payment_type_dim = build_dim(df, ["payment_type"], "payment_type_id")
payment_type_dim["payment_type_name"] = payment_type_dim["payment_type"].map(PAYMENT_TYPE_NAME)
payment_type_dim = payment_type_dim[["payment_type_id", "payment_type", "payment_type_name"]]

payment_type_dim.head()


## Build `fact_table` (same joins & column order)

In [ ]:
fact_table = (
    df.merge(passenger_count_dim, on="passenger_count")
      .merge(trip_distance_dim, on="trip_distance")
      .merge(rate_code_dim, on="RatecodeID")
      .merge(pickup_location_dim, on=["pickup_longitude", "pickup_latitude"])
      .merge(dropoff_location_dim, on=["dropoff_longitude", "dropoff_latitude"])
      .merge(datetime_dim, on=["tpep_pickup_datetime", "tpep_dropoff_datetime"])
      .merge(payment_type_dim, on="payment_type")
)

fact_table = fact_table[
    [
        "VendorID",
        "datetime_id",
        "passenger_count_id",
        "trip_distance_id",
        "rate_code_id",
        "store_and_fwd_flag",
        "pickup_location_id",
        "dropoff_location_id",
        "payment_type_id",
        "fare_amount",
        "extra",
        "mta_tax",
        "tip_amount",
        "tolls_amount",
        "improvement_surcharge",
        "total_amount",
    ]
]

fact_table
